In [10]:
import inro.modeller as m
mm = m.Modeller()
eb = m.Modeller().emmebank
scenario = eb.scenario(1)
network = scenario.get_network()

common = mm.module('org.emme.common')
moe = mm.module('org.emme.moe')

AttributeError: 'Network' object has no attribute 'Mode'

In [71]:
from itertools import islice

def road_MOE(network,eb):    
    vol_attributes = ["@volau1_nt","@volau1_toll","@volau2_nt_hov",
                      "@volau2_nt_nh","@volau2_toll_hov","@volau2_toll_nh",
                      "@volau_htruck","@volau_ltruck","@volau_mtruck"]
    cong_frac =0.85
    
    #VKT/PKT on arterials
    total_VKT_SOV_art = 0.0
    total_VKT_HOV_art = 0.0
    total_PKT_transit_art = 0.0
    total_VKT_trucks_art = 0.0
    
    #VKT/PKT on freeways
    total_VKT_SOV_fwy = 0.0
    total_VKT_HOV_fwy = 0.0
    total_PKT_transit_fwy = 0.0
    total_VKT_trucks_fwy = 0.0
    
    #VHT/PHT on arterials
    total_VHT_SOV_art = 0.0
    total_VHT_HOV_art = 0.0
    total_PHT_transit_art = 0.0
    total_VHT_trucks_art = 0.0
    
    #VHT/PHT on freeways
    total_VHT_SOV_fwy = 0.0
    total_VHT_HOV_fwy = 0.0
    total_PHT_transit_fwy = 0.0
    total_VHT_trucks_fwy = 0.0
    
    #total kilometres
    total_km_sov_fwy = 0.0
    total_km_hov_fwy = 0.0
    total_km_sov_art = 0.0
    total_km_hov_art = 0.0
    
    #total congested kilometres
    cong_km_sov_fwy = 0.0
    cong_km_hov_fwy = 0.0
    cong_km_sov_art = 0.0
    cong_km_hov_art = 0.0


    freeway_vdfs = [11,12,13,14,15,16]

    for link in network.links():
        #skip connector links, transit-only links and links with no length
        if link.volume_delay_func != 90 and link.volume_delay_func !=0 and link.length > 0:
            
            volau = 0.0
            vol_sov = 0.0
            vol_hov = 0.0
            vol_transit = 0.0
            vol_trucks = 0.0
            
            for volume in vol_attributes:
                volau += link[volume]
                #SOV
                if volume in ["@volau1_nt","@volau1_toll"]:
                    vol_sov += link[volume]
                #HOV
                elif volume in ["@volau2_nt_hov","@volau2_nt_nh","@volau2_toll_hov","@volau2_toll_nh"]:
                    vol_hov += link[volume]
                #Trucks
                elif volume in ["@volau_htruck","@volau_ltruck","@volau_mtruck"]:
                    vol_trucks += link[volume]
            
            #transit 
            for line in link.segments():
                 vol_transit  += line["@voltr_avg"]
            
            #calculate link travel time from VDF, in hours
            timau = calc_link_time(link,volau,eb) / 60 
            
            #check if freeway
            if link.volume_delay_func in freeway_vdfs:
                
                total_VKT_SOV_fwy += link.length * vol_sov
                total_VKT_HOV_fwy += link.length * vol_hov
                total_PKT_transit_fwy += link.length * vol_transit
                total_VKT_trucks_fwy += link.length * vol_trucks
                
                total_VHT_SOV_fwy += timau * vol_sov
                total_VHT_HOV_fwy += timau * vol_hov
                total_PHT_transit_fwy += timau * vol_transit
                total_VHT_trucks_fwy += timau * vol_trucks
                
                #check if hov lane
                if network.mode('j') not in link.modes:
                    total_km_hov_fwy += link.length
                    #check if congested
                    if volau/(link.num_lanes*link.data3) > cong_frac:
                        cong_km_hov_fwy += link.length
                else:
                    total_km_sov_fwy += link.length
                    #check if congested
                    if volau/(link.num_lanes*link.data3) > cong_frac:
                        cong_km_sov_fwy += link.length            
            else:
                total_VKT_SOV_art += link.length * vol_sov
                total_VKT_HOV_art += link.length * vol_hov
                total_PKT_transit_art += link.length * vol_transit
                total_VKT_trucks_art += link.length * vol_trucks                
                
                total_VHT_SOV_art += timau * vol_sov
                total_VHT_HOV_art += timau * vol_hov
                total_PHT_transit_art += timau * vol_transit
                total_VHT_trucks_art += timau * vol_trucks
                
                #check if hov lane
                if network.mode('j') not in link.modes:
                    total_km_hov_art += link.length
                    #check if congested
                    if volau/(link.num_lanes*link.data3) > cong_frac:
                        cong_km_hov_art += link.length
                else:
                    total_km_sov_art += link.length
                    #check if congested
                    if volau/(link.num_lanes*link.data3) > cong_frac:
                        cong_km_sov_art += link.length 
                

    print "Total Auto VKT is: " + "{0:,.0f}".format(total_VKT_SOV_art + total_VKT_HOV_art + total_VKT_SOV_fwy + total_VKT_HOV_fwy)
    print "Total Auto VHT is: " + "{0:,.0f}".format(total_VHT_SOV_art + total_VHT_HOV_art + total_VHT_SOV_fwy + total_VHT_HOV_fwy)
    
    print "Average SOV Freeway Speed is: " + "{0:,.2f}".format(total_VKT_SOV_fwy/total_VHT_SOV_fwy)
    print "Average HOV Freeway Speed is: " + "{0:,.2f}".format(total_VKT_HOV_fwy/total_VHT_HOV_fwy)
    print "Average Transit Freeway Speed is: " + "{0:,.2f}".format(total_PKT_transit_fwy/total_PHT_transit_fwy)
    print "Average Truck Freeway Speed is: " + "{0:,.2f}".format(total_VKT_trucks_fwy/total_VHT_trucks_fwy)
   
    print "Average SOV non-freeway Speed is: " + "{0:,.2f}".format(total_VKT_SOV_art/total_VHT_SOV_art)
    print "Average HOV non-Freeway Speed is: " + "{0:,.2f}".format(total_VKT_HOV_art/total_VHT_HOV_art)
    print "Average Transit non-Freeway Speed is: " + "{0:,.2f}".format(total_PKT_transit_art/total_PHT_transit_art)
    print "Average Truck non-Freeway Speed is: " + "{0:,.2f}".format(total_VKT_trucks_art/total_VHT_trucks_art)
    
    print "{0:,.0f}".format(cong_km_sov_fwy/total_km_sov_fwy*100)+"% of SOV freeways are congested"
    print "{0:,.0f}".format(cong_km_hov_fwy/total_km_hov_fwy*100)+"% of HOV freeways are congested" 
    print "{0:,.0f}".format(cong_km_sov_art/total_km_sov_art*100)+"% of SOV non-freeways are congested" 
    print "{0:,.0f}".format(cong_km_hov_art/total_km_hov_art*100)+"% of HOV non-freeways are congested" 



def calc_link_time(link, volau, eb):
    function_string = eb.function("fd" + str(link.volume_delay_func)).expression
    function_string = function_string.replace("length","link.length")
    function_string = function_string.replace("lanes","link.num_lanes")
    function_string = function_string.replace("ul2","link.data2")
    function_string = function_string.replace("ul3","link.data3")
    function_string = function_string.replace("^","**")
    function_string = function_string.replace("(volau+volad)",str(volau))
    return eval(function_string)

road_MOE(network,eb)

Total Auto VKT is: 18,382,190
Total Auto VHT is: 373,359
Average SOV Freeway Speed is: 62.65
Average HOV Freeway Speed is: 62.91
Average Transit Freeway Speed is: 60.72
Average Truck Freeway Speed is: 70.13
Average SOV non-freeway Speed is: 44.42
Average HOV non-Freeway Speed is: 43.15
Average Transit non-Freeway Speed is: 35.35
Average Truck non-Freeway Speed is: 53.65
20% of SOV freeways are congested
18% of HOV freeways are congested
6% of SOV non-freeways are congested
1% of HOV non-freeways are congested


In [45]:
#Reduction in Auto VKT (overall or per capita)
#Reduction in Auto VKT (overall or per capita)
#Green house gas emissions per capita
#Reduction in Auto VHT (overall or per capita)
#Average travel speeds (including trucks and transit on road links)
#% of road network congested
link = network.link(42823,43419)
link.volume_delay_func
for function in eb.functions():
    print function.id, function.expression

fd1 (length*60/ul2/0.98)
fd2 (length*60/ul2)*(1+put((volau+volad)/(lanes*ul3))^4)*(get(1).le.1)+(4*get(1)-2)*(get(1).gt.1)
fd3 (length*60/ul2)*(1+put((volau+volad)/(lanes*ul3))^6)*(get(1).le.1)+(6*get(1)-4)*(get(1).gt.1)
fd8 (length*60/ul2)*((1+put((volau+volad+el1)/(lanes*ul3))^6)*(get(1).le.1)+(6*get(1)-4)*(get(1).gt.1))
fd9 (length*60/ul2)*((1+put((volau+volad+el1)/(lanes*ul3))^6)*(get(1).le.1)+(6*get(1)-4)*(get(1).gt.1))
fd10 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd11 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd12 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd13 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd14 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd15 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd16 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd17 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd18 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd19 (length*60/ul2)*(1+((volau+volad)/(lanes*ul3))^6)
fd20 (length*60/ul2)*

In [21]:
#Route-km of MANAGED lanes
num_of_regions = 24
running_total = [0.0]*num_of_regions
for link in network.links():
    if network.mode("j") not in link.modes and network.mode("c") in link.modes:
        region = int(str(link.type)[0])
        running_total[region] += link.length
#print "There are " + "{0:.2f}".format(running_total) + " km of managed lanes in the network."
index = 1 
for region in running_total:
    print "There are " + "{0:.2f}".format(region) + " km of managed lanes in region " + str(index)
    index += 1
    
    # inter-regional HOV corridor connections to Outer Ring municipalities
# inter-regional HOV / managed lane corridor road connections
# Outer Ring municipalities with no inter-regional HOV / managed lane connection

There are 0.00 km of managed lanes in region 1
There are 18.69 km of managed lanes in region 2
There are 17.27 km of managed lanes in region 3
There are 165.31 km of managed lanes in region 4
There are 102.02 km of managed lanes in region 5
There are 136.58 km of managed lanes in region 6


In [5]:

types = []
for link in network.links():
    if link.type not in types:
        types.append(link.type)
print sorted(types)

[1, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 217, 218, 219, 220, 221, 222, 223, 224, 325, 326, 327, 328, 329, 330, 331, 332, 333, 434, 435, 436, 437, 500, 537, 538, 539, 540, 641, 642, 643, 644, 645, 646, 700, 777, 800, 810, 820, 850, 870, 880, 900, 910, 930, 950]
